# 主要使用 selenium 運作

Doc:https://pypi.org/project/selenium/

Web Driver Download(Chrome):https://sites.google.com/a/chromium.org/chromedriver/downloads

In [1]:
import time
import random
import json
from functools import partial

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
def create_html_with_iframe(iframe):
    """ Generate a html text with iframe
    Args:
        iframe(str): The embbed src from Google Slides
    
    Return:
        str: The html contains iframe.
    
    """
    
    basic_html_with_iframe = """
    <!DOCTYPE html>
    <html>
    <head>
    <meta charset="UTF-8">
    <title>Title of the document</title>
    </head>

    <body>
        %s
    </body>

    </html>

    """%iframe
    
    return basic_html_with_iframe

In [3]:
def register_danmu_to_driver(driver):
    with open("./jquery.danmu.js/jquery-3.4.0.min.js", 'r') as jquery_js: 
        # 3) Read the jquery from a file
        js_text = jquery_js.read() 
        # 4) Load jquery lib
        driver.execute_script(js_text)
    
    with open("./jquery.danmu.js/src/jquery.danmu.js", 'r') as danmu_js: 
        # 3) Read the jquery from a file
        js_text = danmu_js.read() 
        # 4) Load jquery lib
        driver.execute_script(js_text)
        
# document.getElementById("browserLayerViewId").appendChild(new_div);
# document.body.appendChild(new_div);
#     document.querySelector(".punch-viewer-svgpage-svgcontainer").appendChild(new_div);
#     document.getElementsByClassName("punch-viewer-svgpage-svgcontainer")[0].appendChild(new_div);
    
    ADD_DANMU_DIV="""var new_div = document.createElement('div');
    document.body.appendChild(new_div);
    new_div.id = 'danmu';
    """
    driver.execute_script(ADD_DANMU_DIV)    
    INIT = """$("#danmu").danmu({
    height: "100%",  //弹幕区高度
    width: "100%",   //弹幕区宽度
    zindex :100,   //弹幕区域z-index属性
    speed:7000,      //滚动弹幕的默认速度，这是数值值得是弹幕滚过每672像素所需要的时间（毫秒）
    sumTime:65535,   //弹幕流的总时间
    danmuLoop:false,   //是否循环播放弹幕
    defaultFontColor:"#FFFFFF",   //弹幕的默认颜色
    fontSizeSmall:40,     //小弹幕的字号大小
    FontSizeBig:60,       //大弹幕的字号大小
    opacity:"0.9",			//默认弹幕透明度
    topBottonDanmuTime:6000,   // 顶部底部弹幕持续时间（毫秒）
    SubtitleProtection:false,     //是否字幕保护
    positionOptimize:false,         //是否位置优化，位置优化是指像AB站那样弹幕主要漂浮于区域上半部分

    maxCountInScreen: 40,   //屏幕上的最大的显示弹幕数目,弹幕数量过多时,优先加载最新的。
    maxCountPerSec: 10      //每分秒钟最多的弹幕数目,弹幕数量过多时,优先加载最新的。
    });"""
    DANMU_START = "$('#danmu').danmu('danmuStart');"
    driver.execute_script(INIT)
    driver.execute_script(DANMU_START)
    

In [4]:
def send_danmu_to_driver(driver, text):
    if driver is None:
        return
    while True:
        color = random.sample(["white", "red", "green", "blue", "yellow"], 1)[0]
        size = random.randint(0, 1)
# 
        s = """$("#danmu").danmu("addDanmu", { text:"%s" ,color:"%s",size:%s,position:0,time:$('#danmu').data("nowTime")+1})"""\
            % (text, color, size)
#         print("color =", color, "s =", s)
        driver.execute_script(s)
        yield

# 更換下方 Iframe 以及 tlk_url 之網址
不知道這樣蹭得到多少流量呢

In [5]:
iframe = """
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vSB-eOyYXgilTxKYb6wDlB9Ll4YJSiIxg-RoYUCDBOmTeOgBVxE0rObLzdzXd9mdmlELfIUhkBZGm96/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>
"""

In [6]:
tlk_url = "https://tlk.io/ntustcc"

In [7]:
basic_html = create_html_with_iframe(iframe)

# Start Rendering

In [8]:
present_driver = webdriver.Chrome("./chromedriver")

In [9]:
present_driver.execute_script("document.write('{}')".format(json.dumps(basic_html)))

In [10]:
register_danmu_to_driver(present_driver)

In [11]:
send_danmu = partial(send_danmu_to_driver, present_driver)

In [12]:
next(send_danmu("有問題記得要踴躍發問"))

In [13]:
browser = webdriver.Chrome("./chromedriver")

browser.get(tlk_url)
last_time_stamp = cur_time_stamp = 0
# browser.quit()

In [14]:
chatroom = browser.find_element_by_id("chat")
if chatroom:
    chat = chatroom.find_elements_by_class_name("post-message")
    if chat:
        last_time_stamp = int(chat[-1].get_attribute("data-timestamp"))

In [15]:
while True:
    for chat in chatroom.find_elements_by_class_name("post-message"):
        cur_time_stamp = int(chat.get_attribute("data-timestamp"))
        if cur_time_stamp > last_time_stamp:
            next(send_danmu(chat.text))
            last_time_stamp = cur_time_stamp
    time.sleep(3)


KeyboardInterrupt: 

# 下方無視，跑到上面應該就會無限迴圈了，請到 tlk.io 的網址輸入文字，測試結果！

# Start

In [ ]:
s = DANMU_START
driver.execute_script(s)

# Stop

In [ ]:
s = "$('#danmu').danmu('danmuPause');"
driver.execute_script(s)

# Hide all

In [ ]:
s = "$('#danmu').danmu('hideAll');"
driver.execute_script(s)

In [ ]:
browser.quit()